# Send Requests SGLang Runtime


## Launch a server

This code uses `subprocess.Popen` to start an SGLang server process, equivalent to executing 

```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 \
--api-key sk-None-hello-world --host 0.0.0.0 --log-level warning
```
in your command line and wait for the server to be ready.

In [15]:
import subprocess
import sys
import time
import requests

try:
    import sglang
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sglang[all]"])

server_process = subprocess.Popen(
    [
        "python", "-m", "sglang.launch_server",
        "--model-path", "meta-llama/Meta-Llama-3.1-8B-Instruct",
        "--port", "30000",
        "--api-key", "sk-None-hello-world",
        "--host", "0.0.0.0",
        "--log-level", "error"
    ],
    text=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

while True:
    try:
        response = requests.get(
            'http://localhost:30000/v1/models',
            headers={"Authorization": "Bearer sk-None-hello-world"}
        )
        if response.status_code == 200:
            break
    except requests.exceptions.RequestException:
        time.sleep(1)

print("Server is ready. Proceeding with the next steps.")

Server is ready. Proceeding with the next steps.


## Send a Request

Once the server is running, you can send test requests using curl.

In [16]:
!curl http://localhost:30000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer sk-None-hello-world" \
  -d '{"model": "meta-llama/Meta-Llama-3.1-8B-Instruct", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "What is a LLM?"}]}'

{"id":"16c53e804b434d3e9ed714e9b11a0c5b","object":"chat.completion","created":1729634051,"model":"meta-llama/Meta-Llama-3.1-8B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"LLM stands for Large Language Model. It's a type of artificial intelligence (AI) designed to process and understand human language. LLMs are trained on vast amounts of text data, which enables them to learn patterns, relationships, and context within language.\n\nLarge language models use natural language processing (NLP) techniques to analyze and generate human-like text. They can perform a wide range of tasks, such as:\n\n1. **Language translation**: Converting text from one language to another.\n2. **Text summarization**: Condensing long pieces of text into shorter summaries.\n3. **Question answering**: Providing answers to specific questions based on the context.\n4. **Language generation**: Creating original text, such as articles, stories, or even entire books.\n5. **Sentiment analys

## Using OpenAI Compatible API

SGLang supports OpenAI-compatible APIs. Here are Python examples:

In [17]:
import openai

# Always assign an api_key, even if not specified during server initialization.
# Setting an API key during server initialization is strongly recommended.

client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="sk-None-hello-world")

# Chat completion example

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)
print(response)

ChatCompletion(id='6bf3b99b51b34484879dc6f9f1b148be', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are 3 countries and their capitals:\n\n1. **Country:** Japan\n**Capital:** Tokyo\n\n2. **Country:** Australia\n**Capital:** Canberra\n\n3. **Country:** Brazil\n**Capital:** Brasília', refusal=None, role='assistant', function_call=None, tool_calls=None), matched_stop=128009)], created=1729634052, model='meta-llama/Meta-Llama-3.1-8B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=46, prompt_tokens=49, total_tokens=95, prompt_tokens_details=None))


## Use Embedding Models



In [18]:
server_process.terminate()

# Start a new server process for embedding models
# Equivalent to running this in the shell:
# python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct --port 30010 --api-key sk-None-hello-world --host 0.0.0.0 --is-embedding --log-level error
embedding_process = subprocess.Popen(
    [
        "python", "-m", "sglang.launch_server",
        "--model-path", "Alibaba-NLP/gte-Qwen2-7B-instruct",
        "--port", "30010",
        "--api-key", "sk-None-hello-world",
        "--host", "0.0.0.0",
        "--is-embedding",
        "--log-level", "error"
    ],
    text=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)

while True:
    try:
        response = requests.get(
            'http://localhost:30010/v1/models',
            headers={"Authorization": "Bearer sk-None-hello-world"}
        )
        if response.status_code == 200:
            break
    except requests.exceptions.RequestException:
        time.sleep(1)

print("Embedding server is ready. Proceeding with the next steps.")

Embedding server is ready. Proceeding with the next steps.


## Use Curl

In [19]:
!curl http://localhost:30010/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer sk-None-hello-world" \
  -d '{"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": "Once upon a time"}'

{"data":[{"embedding":[0.0083160400390625,0.0006804466247558594,-0.00809478759765625,-0.0006995201110839844,0.0143890380859375,-0.0090179443359375,0.01238250732421875,0.00209808349609375,0.0062103271484375,-0.003047943115234375,0.0029315948486328125,0.0014810562133789062,0.045623779296875,-0.008514404296875,0.003864288330078125,-0.007251739501953125,-0.0013704299926757812,-0.01515960693359375,0.008148193359375,0.003757476806640625,-0.038848876953125,0.004001617431640625,0.004611968994140625,0.0005698204040527344,0.008148193359375,-0.01336669921875,-0.0009927749633789062,0.01132965087890625,0.0167236328125,-0.00334930419921875,0.001857757568359375,0.0014219284057617188,-0.0029888153076171875,0.0015716552734375,-0.008941650390625,-0.003879547119140625,-0.0038814544677734375,0.0182952880859375,-0.01204681396484375,-0.0013942718505859375,0.004367828369140625,-0.00011664628982543945,-0.0088043212890625,0.01329803466796875,-0.00809478759765625,-0.008819580078125,-0.0153656005859375,-0.001006

## Using OpenAI Compatible API

In [20]:
import openai

client = openai.Client(
    base_url="http://127.0.0.1:30010/v1", api_key="sk-None-hello-world")

# Text embedding example

response = client.embeddings.create(
    model="Alibaba-NLP/gte-Qwen2-7B-instruct",
    input="How are you today",
)
print(response)

CreateEmbeddingResponse(data=[Embedding(embedding=[0.00603485107421875, -0.0190582275390625, -0.01273345947265625, 0.01552581787109375, 0.0066680908203125, -0.0135955810546875, 0.01131439208984375, 0.0013713836669921875, -0.0089874267578125, 0.021759033203125, -0.015289306640625, 0.008331298828125, 0.018402099609375, 0.017608642578125, -0.006961822509765625, 0.00539398193359375, -0.01358795166015625, -0.00879669189453125, 0.014404296875, 0.00020015239715576172, -0.03167724609375, -0.0009555816650390625, 0.019500732421875, -0.003391265869140625, -0.004608154296875, 0.003635406494140625, 0.00635528564453125, 0.0125274658203125, 0.014617919921875, 0.005832672119140625, 0.0020236968994140625, 0.00394439697265625, -0.0231170654296875, 0.005359649658203125, -0.01009368896484375, -0.00687408447265625, 0.0004401206970214844, 0.0122528076171875, -0.0027751922607421875, 0.01380157470703125, 0.01081085205078125, 0.01331329345703125, 0.0063629150390625, 0.01214599609375, -0.019256591796875, -0.020

In [21]:
embedding_process.terminate()